In [1]:
%load_ext autoreload
%autoreload 2

# 1. Bài toán minh họa

![](./images/00.png)<br>


# 3. Xây dựng model

In [2]:
from modules.my_pyspark import *
from modules.my_drawer import MyDrawer

In [3]:
spark = MyPySpark(session=True, sql=True)
drawer = MyDrawer()

## 3.1. Chuẩn bị & chuẩn hóa dữ liệu, xác định input, output

* Đọc dữ liệu

In [4]:
file_path = r'./data/Ecommerce_Customers.csv'

In [5]:
data = spark.readFile(file_path)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



> * Các thuộc tính mơ hồ:
>   * `Avg Session Length`: thời gian trung bình mà khách hàng sử dụng web hoặc app của công ty.
>   * `Time on App`: thời gian dùng trên app
>   * `Time on Website`: thời gian dùng trên web
>   * `Length of Membership`: thời gian mất bao lâu để khách hàng trở thành thành viên của công ty
>   * `Yearly Amount Spent`: số tiền mà khách hàng sẽ **cúng** cho công ty trong năm

In [7]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

* Xác định input và output
  * Các thuộc tính input: `Avg Session Length`, `Time on App`, `Time on Website`, `Length of Membership`.
  * Thuộc tính dự đoán: `Yearly Amount Spent`.

In [8]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
input_features = [
    'Avg Session Length',
    'Time on App',
    'Time on Website',
    'Length of Membership'
]

* Từ các thuộc tính trong `input_features` mình sẽ combine nó lại và tạo ra một vector duy nhất gọi là `features` (dc chỉ định tại tham số **outputCol**).

In [10]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel

In [11]:
assembler = VectorAssembler(inputCols=input_features, outputCol='features') # đây là input

In [12]:
data_pre = assembler.transform(data) # transform `data` để khớp vs `assembler`

* Lúc này ta có thể gọi thuộc tính `features` đại diện cùng lúc cho 4 thuộc tính mà ta đã định nghĩa tại `input_features`.

In [13]:
data_pre.select('features').show(2, False) 

+--------------------------------------------------------------------------+
|features                                                                  |
+--------------------------------------------------------------------------+
|[34.49726772511229,12.65565114916675,39.57766801952616,4.0826206329529615]|
|[31.92627202636016,11.109460728682564,37.268958868297744,2.66403418213262]|
+--------------------------------------------------------------------------+
only showing top 2 rows



* Tạo ra tập dữ liệu cuồi cùng gọi là `final_data` dùng để training model với input là `features` và output là `Yearly Amount Spent`.

In [14]:
final_data = data_pre.select('features', 'Yearly Amount Spent')

In [15]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

## 3.2. Chuẩn bị train/test data

In [16]:
train_data, test_data = final_data.randomSplit((0.7, 0.3)) # 0.7 train và 0.3 test

* Xem thống kê mô tả trên training data và test data.

In [17]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                343|
|   mean|  500.3722352703127|
| stddev|  79.75412742795619|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [18]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                157|
|   mean|  497.0021810928547|
| stddev|   78.5500299295117|
|    min| 302.18954780965197|
|    max|  765.5184619388373|
+-------+-------------------+



> * Dữ liệu train và test gần như tương đương, ko có sự chênh lệch cao về mặt thống kê.
> * Nếu chênh lệch giữa các giá trị thống kê của training data và test data là $\leq 10\%$ thì OKLA.

## 3.3. Xây dựng model

* Tạo model Linear Regression
* Tham số `predictionCol` là giá trị mà model sẽ dự đoán, tức $\widehat{y}$

In [19]:
lr = LinearRegression(featuresCol='features', labelCol='Yearly Amount Spent', predictionCol='predict_Yearly Amount Spent') # 

* Fit model với data và gán model cho một biến nào đó

In [20]:
lrModel = lr.fit(train_data)

* In ra coefficients và intercept

In [21]:
lrModel.coefficients, lrModel.intercept

(DenseVector([25.5944, 39.0547, 0.4791, 61.2584]), -1051.5623194632235)

## 3.4. Đánh giá model vs test data

In [22]:
test_results = lrModel.evaluate(test_data)

* Đánh giá phần dư

In [23]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 10.064797262681282|
|-4.9266001221373585|
|  6.899681876762543|
| -1.938494543972297|
| 18.652850069654107|
| -7.087102456761045|
| -5.452626909541891|
| -9.557893479412257|
|  2.458540044765698|
| -18.05821386999446|
|-2.0804413756164877|
|  16.33929658433027|
|  5.859191815935162|
| -6.740288712681547|
| -3.408701006722822|
|-18.869992788883394|
|   7.88907701349018|
| -5.400418842584656|
|  -9.19971726227044|
|0.25560536609708606|
+-------------------+
only showing top 20 rows



* Đánh giá RMSE

In [24]:
test_results.rootMeanSquaredError

10.01416941214838

* Đánh giá mean squared error

In [25]:
test_results.meanSquaredError

100.28358901520825

* Đánh giá $R^2$

In [26]:
test_results.r2

0.9836426776340267

## 3.5. Đánh giá model vs test data

In [27]:
test_model = lrModel.transform(test_data)
test_model.select('predict_Yearly Amount Spent', 'Yearly Amount Spent').show()

+---------------------------+-------------------+
|predict_Yearly Amount Spent|Yearly Amount Spent|
+---------------------------+-------------------+
|          451.7159449335486|  461.7807421962299|
|         472.42850054912697|  467.5019004269896|
|          487.7389278801302|  494.6386097568927|
|         423.26512580092367|  421.3266312569514|
|          444.9385679582865|  463.5914180279406|
|         425.68984455198506|   418.602742095224|
|          449.4182537194238|  443.9656268098819|
|         418.65241967175007|  409.0945261923378|
|         434.05706568459686| 436.51560572936256|
|          564.0037060113993|  545.9454921414049|
|          543.3070253649448|  541.2265839893283|
|          428.2062530667779| 444.54554965110816|
|         469.40423191161335|  475.2634237275485|
|         510.12817600064204|  503.3878872879605|
|         396.21904599052004|  392.8103449837972|
|          519.9924842925398| 501.12249150365636|
|          465.1031696533082|  472.9922466667984|


## 3.6. Lưu trữ & tải model

* Lưu model

In [28]:
file_path1 = r'./models/Ecommerce_Customers_LinearRegression_0'

In [29]:
lrModel.save(file_path1)

* Tải model

In [30]:
from pyspark.ml.regression import LinearRegressionModel

In [31]:
lrModel2 = LinearRegressionModel.load(file_path1)

## 3.7. Dự đoán dữ liệu mới

In [32]:
unlabeled_data = test_data.select('features')
preditions = lrModel2.transform(unlabeled_data)

In [33]:
preditions.show()

+--------------------+---------------------------+
|            features|predict_Yearly Amount Spent|
+--------------------+---------------------------+
|[30.7377203726281...|          451.7159449335486|
|[30.8364326747734...|         472.42850054912697|
|[30.9716756438877...|          487.7389278801302|
|[31.2606468698795...|         423.26512580092367|
|[31.3123495994443...|          444.9385679582865|
|[31.4474464941278...|         425.68984455198506|
|[31.5257524169682...|          449.4182537194238|
|[31.5261978982398...|         418.65241967175007|
|[31.5316044825729...|         434.05706568459686|
|[31.5702008293202...|          564.0037060113993|
|[31.5761319713222...|          543.3070253649448|
|[31.6098395733896...|          428.2062530667779|
|[31.6548096756927...|         469.40423191161335|
|[31.7242025238451...|         510.12817600064204|
|[31.8124825597242...|         396.21904599052004|
|[31.8164283341993...|          519.9924842925398|
|[31.8512531286083...|         